In [1]:
import requests
import json
import os, re, unicodedata
from datetime import datetime
from pyspark.sql import SparkSession, functions as F
from notebookutils import mssparkutils

StatementMeta(, 64e89561-a105-467d-8b97-4dcaaef22aad, 3, Finished, Available, Finished)

In [2]:
# ---------- Spark ----------
spark = SparkSession.builder.getOrCreate()

StatementMeta(, 64e89561-a105-467d-8b97-4dcaaef22aad, 4, Finished, Available, Finished)

In [3]:
# Seleccionar el Lakehouse destino (bronze)
spark.sql("USE lh_bronze")

StatementMeta(, 64e89561-a105-467d-8b97-4dcaaef22aad, 5, Finished, Available, Finished)

DataFrame[]

In [4]:
headers = {
    "Accept": "application/json; application/vnd.esios-api-v1+json",
    "Content-Type": "application/json",
    "x-api-key": "273695bb4f30b8ab8de054a0410ee2a9d09c97a5006e3a07c27615e6c60888a7"
}

StatementMeta(, 64e89561-a105-467d-8b97-4dcaaef22aad, 6, Finished, Available, Finished)

In [5]:
# 📌 Indicadores con sus nombres
indicators = [
    {"type": "Demanda real", "id": "1293"},
    {"type": "PVPC-T-2-0TD", "id": "1001"},
    {"type": "Precio mercado SPOT Diario", "id": "600"},
    {"type": "Mercado SPOT Diario España", "id": "602"}
]

# 📆 Rango de fechas por año
date_ranges = {
    "2023": ("01-01-2023T00:00", "31-12-2023T23:55"),
    "2024": ("01-01-2024T00:00", "31-12-2024T23:55"),
    "2025": ("01-01-2025T00:00", "14-10-2025T23:55"),
}

# ⏱️ Granularidades
time_truncs = ["hour"]

StatementMeta(, 64e89561-a105-467d-8b97-4dcaaef22aad, 7, Finished, Available, Finished)

In [6]:
# 📂 Carpeta base dentro del Lakehouse (BRONZE)
base_dir = "/lakehouse/default/Files/bronze/ESIOS/data/mercados/componentes-precio"

StatementMeta(, 64e89561-a105-467d-8b97-4dcaaef22aad, 8, Finished, Available, Finished)

In [7]:
def slugify(name: str) -> str:
    # Normalizar y quitar tildes
    name = unicodedata.normalize("NFKD", name)
    name = name.encode("ascii", "ignore").decode("utf-8")  # quita acentos
    # Sustituir caracteres no válidos por "_"
    return re.sub(r'[^a-z0-9_]', '_', name.lower())

StatementMeta(, 64e89561-a105-467d-8b97-4dcaaef22aad, 9, Finished, Available, Finished)

In [8]:
# 🚀 Loop de descargas
for indicator in indicators:
    for year, (start_date, end_date) in date_ranges.items():
        for trunc in time_truncs:
            url = (
                f"https://api.esios.ree.es/indicators/{indicator['id']}?"
                f"start_date={start_date}&end_date={end_date}&time_trunc={trunc}"
            )
            try:
                response = requests.get(url, headers=headers, timeout=60)
                response.raise_for_status()
                data = response.json()

                # ----------------------
                # 1) Guardar JSON crudo en Files/
                # ----------------------
                filename = f"brz-mercados-{slugify(indicator['type'])}-{trunc}-{year}.json"
                year_dir = os.path.join(base_dir, trunc, str(year))
                os.makedirs(year_dir, exist_ok=True)
                filepath = os.path.join(year_dir, filename)

                with open(filepath, "w", encoding="utf-8") as f:
                    json.dump(data, f, ensure_ascii=False, indent=2)

                print(f"✅ Guardado JSON en Lakehouse: {filepath}")

            except requests.exceptions.RequestException as e:
                print(f"❌ Error en {url}: {e}")

print(f"\nProceso terminado - {datetime.utcnow().isoformat()}Z")

StatementMeta(, 64e89561-a105-467d-8b97-4dcaaef22aad, 10, Finished, Available, Finished)

✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/mercados/componentes-precio/hour/2023/brz-mercados-demanda_real-hour-2023.json
✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/mercados/componentes-precio/hour/2024/brz-mercados-demanda_real-hour-2024.json
❌ Error en https://api.esios.ree.es/indicators/1293?start_date=01-01-2025T00:00&end_date=14-10-2025T23:55&time_trunc=hour: 504 Server Error: Gateway Time-out for url: https://api.esios.ree.es/indicators/1293?start_date=01-01-2025T00:00&end_date=14-10-2025T23:55&time_trunc=hour
✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/mercados/componentes-precio/hour/2023/brz-mercados-pvpc_t_2_0td-hour-2023.json
✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/mercados/componentes-precio/hour/2024/brz-mercados-pvpc_t_2_0td-hour-2024.json
❌ Error en https://api.esios.ree.es/indicators/1001?start_date=01-01-2025T00:00&end_date=14-10-2025T23:55&time_tru